In [1]:
import openai 
import config
import os

In [2]:
# openai.api_key = config.API_KEY
os.environ["OPENAI_API_KEY"] = config.API_KEY

# To create a training file

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
file = client.files.create(
  file=open("train_data_prepared.jsonl", "rb"),
  purpose="fine-tune"
)
file_id = file.id
# print(file_id)

In [6]:
job_id= client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
).id
# print(job_id)

# To fine-tune a model on the training file

In [7]:
client.fine_tuning.jobs.list(limit=10).data

[FineTuningJob(id='ftjob-6Azw6sKYd7mZGAN7WOm7PRZ7', created_at=1708694545, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=7, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-tGCVVkvJGer1gXBNYJ5TusdW', result_files=[], status='running', trained_tokens=None, training_file='file-CORhRgXGBeu54ajOQggrXDos', validation_file=None),
 FineTuningJob(id='ftjob-wWg5J89NSZqM4VO3i48FGG1X', created_at=1708694384, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=7, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-tGCVVkvJGer1gXBNYJ5TusdW', result_files=[], status='running', trained_tokens=None, training_file='file-ZDx11xnzd0KiuO3TlZMIc7vz', validation_file=None),
 FineTuningJob

In [8]:
# List 10 fine-tuning jobs
job_num = -1 # choosing oldest model; note: latest created will be assigned index 0; FIFO
item = client.fine_tuning.jobs.list(limit=10).data[job_num]
job_id = item.id
training_file = item.training_file
# print(job_id)
# print(training_file)

In [9]:
fine_tuned_model_name = client.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
# print(fine_tuned_model_name)

In [10]:
print(fine_tuned_model_name)

ft:gpt-3.5-turbo-0613:solarspaceclouds::8u196nio


In [11]:
# client.fine_tuning.jobs.retrieve(job_id)

In [12]:
# from openai import OpenAI
# client = OpenAI()

# # List 10 fine-tuning jobs
# client.fine_tuning.jobs.list(limit=10)

# # Retrieve the state of a fine-tune
# client.fine_tuning.jobs.retrieve("ftjob-abc123")

# # Cancel a job
# client.fine_tuning.jobs.cancel("ftjob-abc123")

# # List up to 10 events from a fine-tuning job
# client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10)

# # Delete a fine-tuned model (must be an owner of the org the model was created in)
# client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

# Use the fine-tuned model

In [13]:
prompt = "Arabian Nights."

In [14]:
completion = client.chat.completions.create(
  # model="ft:gpt-3.5-turbo-0613:solarspaceclouds::8u196nio",
  model = fine_tuned_model_name,
  messages=[
    {"role": "system", "content": f"{prompt}"},
    {"role": "user", "content": "Tell me the book summary"}
  ]
)

In [15]:
print(completion.choices[0].message.content)

Arabian Nights is a collection of ancient Middle Eastern and South Asian stories, compiled and translated over many centuries. The book is framed by the tale of Scheherazade, a clever young woman who marries an unpredictable king. In order to ensure her survival each night, she weaves stories and cliffhangers, captivating the king's interest and delaying her own execution. Through her storytelling, readers are transported to a world of magical genies, mystical creatures, epic adventures, and timeless moral lessons. The collection includes well-known tales such as Aladdin's Wonderful Lamp, Ali Baba and the Forty Thieves, and Sinbad the Sailor, capturing the essence of Arabian folklore and serving as a testament to the power of imagination and the transformative power of storytelling.
